In [1]:
#Dependencies
from bs4 import BeautifulSoup
import requests
import json

In [10]:
#utilts
import os
def printSoupToHtml(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.html'):
        with open('soup.html', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.html', 'w', encoding='utf-8') as file:
        file.write(str(soup))
    return True
def printSoupToJson(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.json'):
        with open('soup.json', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(soup))
    return True

In [19]:
#Functions
def getStringBetweenTwoWords(string, first, last):
    try:
        start = string.index(first) + len(first)
        end = string.index(last, start)
        return string[start:end]
    except ValueError:
        return ""

async def scrapeAliExpress(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    #printSoupToHtml(soup)

    # Assuming 'soup' is the BeautifulSoup object containing the parsed HTML

    # Find the <script> tag containing ACC.config.productDetails
    script_tag = soup.find('script')

    if script_tag:
        # Extract the JavaScript content from the <script> tag
        script_content = script_tag.string
        script_target_object= getStringBetweenTwoWords(script_content, 'window.runParams =', 'window.runParams.is23').replace(';', '')
        script_target_object = script_target_object.replace('data', '"data"')
        # convert the JavaScript object to a Python dictionary
        script_target_object = json.loads(script_target_object)
        
        # Extract price and title from the parsed dictionary
        price = script_target_object['data']['metaDataComponent']['ogTitle'].split('|')[0].strip()
        title = script_target_object['data']['metaDataComponent']['title'].split('|')[0].strip()
        printSoupToJson(script_target_object['data']['imageComponent']['imagePathList'])
        ImageList = script_target_object['data']['imageComponent']['imagePathList']
        # Display the extracted price and title
        print("Price:", price)
        print("Title:", title)
        print("ImageList:", ImageList)


In [20]:

url = 'https://ar.aliexpress.com/item/1005006860843367.html?spm=a2g0o.productlist.main.1.4cbdDLvvDLvvt5&algo_pvid=7246bb87-519c-45f0-9096-37d5a5e6b81c&utparam-url=scene%3Asearch%7Cquery_from%3A'
await scrapeAliExpress(url)

Price: 50998.4EGP
Title: Apple iPhone 15 Pro ، A3104 ، APPLE ، A17 Pro ، Nano SIM ، IP68 ، إصدار CN ، علامة تجارية جديدة ونشطة ، أصلية ، منتجات أصلية
